# S3 storage utility functions

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import stat
import sys

from pprint import pprint

In [3]:
sys.path.append('..')

Dictionary to be saved

In [4]:
house_info = {'city': 'den haag',
              'type': 'apartment',
              'rooms': 3,
              'occupants': ['rod', 'nup']}

### functions

Make sure to have the credentials in `~/.aws/credentials`
```
[defaualt]
aws_access_key_id=
aws_secret_access_key=
```
and if necessary, specify a region in `~/.aws/config` 
```
[default]
region=
```

In [29]:
import boto3

from ufuncs.storage import s3_put_object
from ufuncs.storage import s3_get_object

In [30]:
s3_path = '/godatadriven-sandbox/test.pkl'
s3 = boto3.resource('s3')

save file

In [31]:
s3_put_object(house_info, storage_path=s3_path)

{'ETag': '"4589888fd546fd0555f8c69a85343a4e"',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
   'date': 'Fri, 16 Jun 2017 19:05:06 GMT',
   'etag': '"4589888fd546fd0555f8c69a85343a4e"',
   'server': 'AmazonS3',
   'x-amz-id-2': 'yLe9BiRa7d3NvvK6q40mTnp/qGY7UfHK2yXU/RySVpBvMamyOCacirRsF+D2/HV4+pqczn5tKW0=',
   'x-amz-request-id': '9CCF554CBAE5915B'},
  'HTTPStatusCode': 200,
  'HostId': 'yLe9BiRa7d3NvvK6q40mTnp/qGY7UfHK2yXU/RySVpBvMamyOCacirRsF+D2/HV4+pqczn5tKW0=',
  'RequestId': '9CCF554CBAE5915B',
  'RetryAttempts': 0}}

In [32]:
print('File Permissions:')
pprint(s3.Object('godatadriven-sandbox', 'test.pkl').Acl().grants)

File Permissions:
[{'Grantee': {'ID': '2917aaf5ecd3d435c84b4fc250fc3f1a628413bb8b65c3874d83809faf419106',
              'Type': 'CanonicalUser'},
  'Permission': 'FULL_CONTROL'}]


save and overwrite

In [33]:
s3_put_object(house_info, storage_path=s3_path)

{'ETag': '"4589888fd546fd0555f8c69a85343a4e"',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
   'date': 'Fri, 16 Jun 2017 19:05:11 GMT',
   'etag': '"4589888fd546fd0555f8c69a85343a4e"',
   'server': 'AmazonS3',
   'x-amz-id-2': 'DcH0I1xhGxqaKuZC+yzKLQy1rBLvwQscHdaps3VWyAWQvpdEQ9H3MSKZSfzj0iO8i3q1mZ148R8=',
   'x-amz-request-id': 'AF150ADDA29D5B61'},
  'HTTPStatusCode': 200,
  'HostId': 'DcH0I1xhGxqaKuZC+yzKLQy1rBLvwQscHdaps3VWyAWQvpdEQ9H3MSKZSfzj0iO8i3q1mZ148R8=',
  'RequestId': 'AF150ADDA29D5B61',
  'RetryAttempts': 0}}

In [34]:
print('File Permissions:')
pprint(s3.Object('godatadriven-sandbox', 'test.pkl').Acl().grants)

File Permissions:
[{'Grantee': {'ID': '2917aaf5ecd3d435c84b4fc250fc3f1a628413bb8b65c3874d83809faf419106',
              'Type': 'CanonicalUser'},
  'Permission': 'FULL_CONTROL'}]


save and not overwrite

In [35]:
s3_put_object(house_info, storage_path=s3_path, overwrite=False)

FileExistsError: S3 object '/godatadriven-sandbox/test.pkl' already exists and versioning is Suspended. Argument overwrite is set to False.

save, overwrite and set permission to `public-read`

In [36]:
s3_put_object(house_info, storage_path=s3_path, ACL='public-read')

{'ETag': '"4589888fd546fd0555f8c69a85343a4e"',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
   'date': 'Fri, 16 Jun 2017 19:05:26 GMT',
   'etag': '"4589888fd546fd0555f8c69a85343a4e"',
   'server': 'AmazonS3',
   'x-amz-id-2': 'XFRdRuWPq62vXyuX3lvObAGnwHJhDUB2A4QyCO0OtCzpqQ+HLFkUI3rAF5l7cFSHb0N2v9F0jwk=',
   'x-amz-request-id': '6924239F92ABA7BB'},
  'HTTPStatusCode': 200,
  'HostId': 'XFRdRuWPq62vXyuX3lvObAGnwHJhDUB2A4QyCO0OtCzpqQ+HLFkUI3rAF5l7cFSHb0N2v9F0jwk=',
  'RequestId': '6924239F92ABA7BB',
  'RetryAttempts': 0}}

In [37]:
print('File Permissions:')
pprint(s3.Object('godatadriven-sandbox', 'test.pkl').Acl().grants)

File Permissions:
[{'Grantee': {'ID': '2917aaf5ecd3d435c84b4fc250fc3f1a628413bb8b65c3874d83809faf419106',
              'Type': 'CanonicalUser'},
  'Permission': 'FULL_CONTROL'},
 {'Grantee': {'Type': 'Group',
              'URI': 'http://acs.amazonaws.com/groups/global/AllUsers'},
  'Permission': 'READ'}]


reading object

In [39]:
s3_get_object(s3_path)

{'city': 'den haag',
 'occupants': ['rod', 'nup'],
 'rooms': 3,
 'type': 'apartment'}

delete file

In [40]:
s3.Object('godatadriven-sandbox', 'test.pkl').delete()

{'DeleteMarker': True,
 'ResponseMetadata': {'HTTPHeaders': {'date': 'Fri, 16 Jun 2017 19:05:57 GMT',
   'server': 'AmazonS3',
   'x-amz-delete-marker': 'true',
   'x-amz-id-2': 'Fu9iRlwPJW5QcAM+22kcofKRxwyzI5SZV6Psq8edPUQdfS+nCLuQhMZMH7c2p5H6EbRkqN2bcSA=',
   'x-amz-request-id': '7FF0BED65C75DC19',
   'x-amz-version-id': 'null'},
  'HTTPStatusCode': 204,
  'HostId': 'Fu9iRlwPJW5QcAM+22kcofKRxwyzI5SZV6Psq8edPUQdfS+nCLuQhMZMH7c2p5H6EbRkqN2bcSA=',
  'RequestId': '7FF0BED65C75DC19',
  'RetryAttempts': 0},
 'VersionId': 'null'}